# Part 1

In [1]:
with open("inputs/day9.txt", 'r') as fn:
    input = fn.read()

In [2]:
import pandas as pd

In [3]:
def part1(input):
    lines = [pd.Series(map(int, l.split(" "))) for l in input.splitlines()]
    forecasts = list()
    for series in lines:
        diffs = [series]
        while diffs[-1].nunique() > 1:
            diffs.append(diffs[-1].diff())
        forecasts.append(pd.concat(diffs,axis=1).iloc[-1].sum())
    print(int(sum(forecasts)))

In [4]:
part1(input)

1992273652


# Part 2

In [5]:
def part2(input):
    lines = [pd.Series(map(int, l.split(" "))) for l in input.splitlines()]
    forecasts = list()
    for series in lines:
        diffs = [series[::-1]]
        while diffs[-1].nunique() > 1:
            diffs.append(diffs[-1].diff())
        s = pd.concat(diffs,axis=1).iloc[-1]
        forecasts.append(s.sum())
    print(int(sum(forecasts)))

In [6]:
part2(input)

1012


# SKTime

In [7]:
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.naive import NaiveForecaster
from sktime.transformations.series.difference import Differencer


def sktime_sol(series, diff_order):
    pipe = TransformedTargetForecaster(steps=[
        ("differencer", Differencer(lags=[1]*diff_order)),
        ("forecaster", NaiveForecaster(strategy="last")),
    ])
    return pipe.fit_predict(series, fh=[1])

In [8]:
def sktime(input):
    lines = [pd.Series(map(int, l.split(" "))) for l in input.splitlines()]
    next_forecasts = list()
    prev_forecasts = list()
    
    for series in lines:
        diffs = [series]
        while diffs[-1].nunique() > 1:
            diffs.append(diffs[-1].diff())
        
        forecast_fwd = sktime_sol(series, len(diffs))
        forecast_back = sktime_sol(series[::-1].reset_index(drop=True), len(diffs))

        next_forecasts.append(forecast_fwd)
        prev_forecasts.append(forecast_back)
    
    print("Part 1: ", int(sum(next_forecasts).tolist()[0]))
    print("Part 2: ", int(sum(prev_forecasts).tolist()[0]))


In [9]:
sktime(input)

Part 1:  1992273652
Part 2:  1012
